# **Въведение в NumPy**

## **Зад 1. Тригонометрични функции към градуси** 

Допишете кода, така че да се изчислява синус, косинус и тангенс на ъглите, зададени в градуси.

In [1]:
import numpy as np

arr = np.array([0., 30., 45., 60., 90.])

print(np.sin(arr) * np.pi / 180.)
print(np.cos(arr) * np.pi / 180.)
print(np.tan(arr) * np.pi / 180.)

[ 0.         -0.0172444   0.01485107 -0.00531995  0.01560319]
[ 0.01745329  0.0026922   0.0091686  -0.01662274 -0.00782036]
[ 0.         -0.11179412  0.02827041  0.00558576 -0.03482282]


## **Зад 2. Дискретно Фурие преобразуване**

Напишете функция **dft()**, която изчислява дискретното Фурие преобразуване на вектор с комплексни числа чрез дефиницията:

$$
X_k = \sum_{n=0}^{N-1} x_n \cdot e^{-2 \pi i \frac{k n}{N}}, \quad k = 0, 1, \dots, N-1
$$


In [2]:
import numpy as np

def dft(x):
    N = len(x)
    X = np.zeros(N, dtype=complex)
    for k in range(N):
        for n in range(N):
            X[k] += x[n] * np.exp(-2j * np.pi * k * n / N)
    return X

x = np.array([1+1j, 2+0j, 0-1j])
print(dft(x))

[ 3.       +0.j          0.8660254-0.23205081j -0.8660254+3.23205081j]


- "Разглобяваме" един сигнал (функция) на "строителните му блокове", в случая комплексните експоненти т.е. от кои честоти е направен сигнала и колко силно пприсъства всяка от тях. *Например*, изсвирваме музикален акорд на китара, значи имаме смесица от няколко ноти. Фурие трансформацията казва следното: в този акорд има 440 Hz (Ла) и с каква сила е;
- Дискрестно е, защото в компютрите често работим с дискретни функции, а **не** с непрекъснати.

## **Зад 3. Simplex Method**

Алгоритъм за решаване на линейни оптимизационни задачи от типа:

$$
\text{maximize } c^T x \quad \text{subject to } Ax \le b, \quad x \ge 0
$$

където: `x` е вектор на променливите, `A` е матрица на коефициентите, `b` е вектор на ограниченията, `c` е вектор на коефициентите на целевата функция.

Области от възможни решения (системата `Ax<=b` и `x>=0`) образуват многоъгълник. Линейната функция `c^T.x` достига максимума си върху някой от върховете на многоъгълника. За всяка итерация се избират `m` базисни променливи (където `m` е броят на ограниченията). Остатъчните `n-m` променливи са небазисни и се задават на нула. Изчислява се текущото решение, като се решава системата `B.x_B = b`, където `B` е матрицата на колоните за базисните променливи, а `x_B` са стойностите им. Проверяват се `редуцираните разходи` за небазисните променливи. Ако всички са `>=0`, текущото решение е оптимално. Ако има отрицателни редуцирани разходи, избира се променлива за влизане в базиса. Определя се коя базисна променлива излиза от базиса чрез `правилото на минималното съотношение`. Актуализира базиса и преминава към следващата итерация.

Ако имаме следното условие:
$$ max Z = 3x_1 + 2x_2 $$
$$
\begin{aligned}
x_1 + x_2 &\le 4,\\
2x_1 + x_2 &\le 5,\\
x_1, x_2 &\ge 0.
\end{aligned}
$$
Попълнете в main функцията `c`, `A` и `b` и след това намерете грешките в кода.

In [3]:
import numpy as np

def simplex(c, A, b):
    # Конвертиране в канонична форма
    m, n = A.shape
    A = np.hstack([A, np.eye(m)])
    c = np.concatenate([c, np.zeros(m)])
    
    basis = list(range(n, n + m))
    non_basis = list(range(n))
    
    tableau = np.vstack([
        np.hstack([A, b.reshape(-1, 1)]),
        np.hstack([-c, 0])
    ])
    
    while True:
        if np.all(tableau[-1, :-1] >= 0): # оптималността не е <= 0
            break
            
        entering = np.argmin(tableau[-1, :-1])
        
        if np.all(tableau[:-1, entering] <= 0): # неограничеността не е >= 0
            raise ValueError("Problem is unbounded")
            
        ratios = []
        for i in range(m):
            if tableau[i, entering] > 0:
                ratios.append(tableau[i, -1] / tableau[i, entering])
            else:
                ratios.append(np.inf)
        leaving = np.argmin(ratios)

        pivot = tableau[leaving, entering]
        tableau[leaving] /= pivot
        
        for i in range(m + 1):
            if i != leaving:
                tableau[i] -= tableau[i, entering] * tableau[leaving]
        
        basis[leaving] = entering
    
    x = np.zeros(n + m)
    for i in range(m):
        if basis[i] < n:
            x[basis[i]] = tableau[i, -1]
    
    return x[:n], -tableau[-1, -1]

if __name__ == "__main__":
    c = np.array([3, 2])      # Целева функция
    A = np.array([[1, 1],     # Ограничения
                  [2, 1]])
    b = np.array([4, 5])      # Десни части
    
    solution, optimal_value = simplex(c, A, b)
    print(f"Оптимално решение: x1 = {solution[0]:.2f}, x2 = {solution[1]:.2f}")
    print(f"Оптимална стойност: {optimal_value:.2f}")

Оптимално решение: x1 = 1.00, x2 = 3.00
Оптимална стойност: -9.00
